### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="bbr-trace-validation-" + fablib.get_bastion_username()
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)
     

#slice = fablib.new_slice(name=slice_name)

In [ ]:
n_senders = 10
n_receivers = 1

In [ ]:
exp_requires = {'core': (n_senders+n_receivers)*4 + 4, 'nic': (n_senders + n_receivers + 1)*1}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic'])  ):
        break

fablib.show_site(site_name)

In [ ]:
# this cell sets up the hosts
slice.add_node(name='router', site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_22')

sender_names = ["sender-"+str(i) for i in range(n_senders)]
for n in sender_names:
    slice.add_node(name=n, site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_22')  

receiver_names = ["receiver-"+str(i) for i in range(n_receivers)]
for n in receiver_names:
    slice.add_node(name=n, site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_22')  



In [ ]:
# this cell sets up the network links
nets = [
    {"name": "link-sender",    "nodes": sender_names,  "idx": 0},
    {"name": "link-receiver",  "nodes": receiver_names, "idx": 1}
]

router_iface = []
router_iface.append(slice.get_node('router').add_component(model="NIC_Basic", name='link-sender').get_interfaces()[0])
router_iface.append(slice.get_node('router').add_component(model="NIC_Basic", name='link-receiver').get_interfaces()[0])
print(router_iface)



In [ ]:
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ] + [router_iface[n["idx"]]]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)


In [ ]:
slice.submit(wait_timeout=5000)

In [ ]:
slice.get_state()

In [ ]:
slice.wait_ssh(progress=True)

## Install BBRv2 kernel

In [ ]:
pkg_list = ['linux-headers-5.13.12_5.13.12-2_amd64.deb',
            'linux-libc-dev_5.13.12-2_amd64.deb',
            'linux-image-5.13.12_5.13.12-2_amd64.deb']

cmd_BBRv2 ="""sudo grub-set-default "Advanced options for Ubuntu>Ubuntu, with Linux 5.13.12"
sudo grub-mkconfig -o /boot/grub/grub.cfg
sudo sed -i 's/^GRUB_DEFAULT=.*/GRUB_DEFAULT=saved/' /etc/default/grub
sudo update-grub
sudo reboot"""

for n in sender_nodes:
    for pkg in pkg_list:
        n.execute("wget https://raw.githubusercontent.com/sdatta97/imcbbrrepro/main/setup/{packet}".format(packet=pkg))
    n.execute("sudo dpkg -i  *.deb")
    n.execute(cmd_BBRv2)


In [ ]:
pkg_list = ['linux-headers-5.13.12_5.13.12-2_amd64.deb',
            'linux-libc-dev_5.13.12-2_amd64.deb',
            'linux-image-5.13.12_5.13.12-2_amd64.deb']

cmd_BBRv2 ="""sudo grub-set-default "Advanced options for Ubuntu>Ubuntu, with Linux 5.13.12"
sudo grub-mkconfig -o /boot/grub/grub.cfg
sudo sed -i 's/^GRUB_DEFAULT=.*/GRUB_DEFAULT=saved/' /etc/default/grub
sudo update-grub
sudo reboot"""

for n in receiver_nodes:
    for pkg in pkg_list:
        n.execute("wget https://raw.githubusercontent.com/sdatta97/imcbbrrepro/main/setup/{packet}".format(packet=pkg))
    n.execute("sudo dpkg -i  *.deb")
    n.execute(cmd_BBRv2)

In [ ]:
for n in sender_nodes:
    n.execute("uname -a")

for n in receiver_nodes:
    n.execute("uname -a")

In [ ]:
for n in sender_nodes:
    n.execute("sudo modprobe tcp_bbr2")

## Configure resources

In [ ]:
slice = fablib.get_slice(name=slice_name)

Bring up all of the network interfaces:

In [ ]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

Assign addresses to router interfaces and enable forwarding:

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_sender = slice.get_interface('router-link-sender-p1')
if_sender.ip_addr_add(addr="10.10.1.1", subnet=IPv4Network("10.10.1.0/24"))
if_receive = slice.get_interface('router-link-receiver-p1')
if_receive.ip_addr_add(addr="10.10.2.1", subnet=IPv4Network("10.10.2.0/24"))

slice.get_node(name='router').execute("sudo sysctl -w net.ipv4.ip_forward=1")

Assign addresses to host (sender and receiver) interfaces and set up routes:

In [ ]:
for i in range(n_senders):
    if_name = slice.get_interface('sender-' + str(i) + '-link-sender-p1')
    if_name.ip_addr_add(addr="10.10.1.1" + str(i) , subnet=IPv4Network("10.10.1.0/24"))
    slice.get_node(name='sender-' + str(i)).ip_route_add(subnet=IPv4Network("10.10.2.0/24"), gateway="10.10.1.1")
    
for i in range(n_receivers):
    if_name = slice.get_interface('receiver-' + str(i) + '-link-receiver-p1')
    if_name.ip_addr_add(addr="10.10.2.1" + str(i) , subnet=IPv4Network("10.10.2.0/24"))
    slice.get_node(name='receiver-' + str(i)).ip_route_add(subnet=IPv4Network("10.10.1.0/24"), gateway="10.10.2.1")

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

Also install `iperf3` on sender and receiver hosts:

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_senders)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_receivers)]

In [ ]:
from ipaddress import ip_address, IPv6Address
for n in sender_nodes:
    n.execute("sudo apt update; sudo apt -y install iperf3; sudo apt -y install python3; sudo modprobe tcp_bbr")
    n.execute_thread("sudo modprobe tcp_bbr")
              
for n in receiver_nodes:
    n.execute_thread("sudo apt update; sudo apt -y install iperf3")
    


In [ ]:
for n in sender_nodes:
    n.execute("sudo apt -y install python3")

In [ ]:
for n in sender_nodes:
    n.execute("iperf3 -version")

In [ ]:
for n in sender_nodes:
    n.execute("sudo apt-get update; sudo apt-get install -y python3-pip ethtool netcat moreutils ",quiet=True)
    n.execute("sudo python3 -m pip install scikit-learn numpy pandas matplotlib seaborn", quiet=True)
    
for n in receiver_nodes:
    n.execute("sudo apt-get update; sudo apt-get install -y python3-pip ethtool netcat moreutils ",quiet=True)
    n.execute("sudo python3 -m pip install scikit-learn numpy pandas matplotlib seaborn", quiet=True)

## Draw the network topology 

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
all_nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(all_nodes),len(all_nodes)))
G = nx.Graph()
G.add_nodes_from(all_nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in all_nodes], 
        node_size=[len(n[0])*400 for n in all_nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);
     

## Extend your slice

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

## Log in to hosts

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_senders)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_receivers)]
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

sender_egress_interfaces = [slice.get_node(name='sender-' + str(i)).get_interface(network_name = "link-sender")  for i in range(n_senders)]
sender_egress_names = [iface.get_device_name() for iface in sender_egress_interfaces]

receiver_ingress_interfaces = [slice.get_node(name='receiver-' + str(i)).get_interface(network_name = "link-receiver")  for i in range(n_receivers)]
receiver_ingress_names = [iface.get_device_name() for iface in receiver_ingress_interfaces]

In [ ]:
print(sender_egress_interfaces)

## Configure queues

In [ ]:
import math

btl_rate = 100 # in Mbps

btl_link_delay = 10 # in ms

limit = math.ceil((1*btl_rate * btl_link_delay * 10^3)/8)

avpkt = 1000

print(limit)

print(math.ceil(limit*5/(36*avpkt)))


router_node.execute(f"sudo tc qdisc del dev {router_egress_name} root")
router_node.execute(f"sudo tc qdisc replace dev {router_egress_name} root handle 1: htb default 3")
router_node.execute(f"sudo tc class add dev {router_egress_name} parent 1: classid 1:3 htb rate {btl_rate}mbit")

router_node.execute(f"sudo tc qdisc add dev {router_egress_name} parent 1:3 handle 3: red limit {limit} max {limit} avpkt 1000 bandwidth {btl_rate}Mbit burst 1")

In [ ]:
import random
for n in sender_nodes:
    sender_inf = n.get_interface(network_name="link-sender")
    sender_inf_name = sender_inf.get_device_name()
    
    sender_delay = round(random.uniform(20,30),1)
    
    print(sender_delay)
    
    n.execute(f"sudo tc qdisc del dev {sender_inf_name} root")
    n.execute(f"sudo tc qdisc add dev {sender_inf_name} root netem delay {sender_delay}ms limit 1000000")
    # n.execute(f"sudo tc qdisc add dev {sender_inf_name} parent 1:1 handle 10: fq pacing")

In [ ]:
for n in sender_nodes:
    n.execute("sudo modprobe tcp_bbr")
    n.execute("sudo modprobe tcp_bbr2")
    
for n in receiver_nodes:
    n.execute("sudo modprobe tcp_bbr")
    n.execute("sudo modprobe tcp_bbr2")

### Execute Experiment

In [ ]:
receiver_nodes[0].execute("rm -r scherrer-exp-aggregate")
router_node.execute("rm -r scherrer-exp-aggregate")

In [ ]:
# Set experimental parameters : generate full factorial experiment
import itertools
exp_factors = {
    'rate': [100],
    'btl_delay': [10],
    'cca': ["bbr-bbr"],
    'test_duration': [120],
    'flows': [1],
    'interval': [0.1],
    'omit': [180],
    'aqm': ['FIFO'],
    'buffer_factor' : [1],
    'trial': [1,2,3]
}

factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))
exp_lists = [dict(zip(factor_names, factor_l)) for factor_l in factor_lists]
#'cca': ["bbr-bbr","bbr-bbr2","bbr-cubic","bbr-reno","bbr2-bbr2","bbr2-cubic","bbr2-reno"],

In [ ]:
import os
import csv

os.remove('experiment_results.csv')
# Create a list of header fields for the CSV file
fields = ['rate', 'btl_delay', 'cca', 'test_duration', 'flows', 'interval', 'omit', 'aqm', 'buffer_factor', 'trial', 'jfi', 'loss_percentage', 'utilization', 'buffer_occupancy']

# Write the header to the CSV file
with open('experiment_results.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()

In [ ]:
fields = ['rate', 'btl_delay', 'cca', 'test_duration', 'flows', 'interval', 'omit', 'aqm', 'buffer_factor', 'trial', 'jfi', 'loss_percentage', 'utilization', 'buffer_occupancy']

In [ ]:
# Set up router queue

router_node.execute(f"sudo tc qdisc del dev {router_egress_name} root")
router_node.execute(f"sudo tc qdisc replace dev {router_egress_name} root handle 1: htb default 3")
router_node.execute(f"sudo tc class add dev {router_egress_name} parent 1: classid 1:3 htb rate 100Mbit")

In [ ]:
import time
import os
import csv
import random
import math
import itertools

router_script="rm queue.txt; start_time=$(date +%s); while true; do tc -p -s -d qdisc show dev {iface} | tr -d \'\n\' | ts '%.s' | tee -a queue.txt; echo \"\" | tee -a queue.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.01; done;"

def create_directory(directory_name):
    if not os.path.exists(directory_name):
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    else:
        print(f"Directory '{directory_name}' already exists.")
        
        
def setup_experiment(exp):
    directory_name = "scherrer-exp-aggregate"
    create_directory(directory_name)

    data_dir = os.path.join(os.getcwd(), directory_name)
    iequal_file_out = os.path.join(data_dir, f'ieq_{"_".join(str(v) for v in exp.values())}.csv')

    if os.path.exists(iequal_file_out):
        print("Already have relevant files, skipping")
        return

    print("Running experiment for exp:", iequal_file_out)
    
    limit = math.ceil(((exp['buffer_factor'])*exp['rate'] * exp['btl_delay'] * 10**3)/8)
    
    if exp['aqm'] == 'FIFO':
        router_node.execute(f"sudo tc qdisc replace dev {router_egress_name} parent 1:3 handle 3: bfifo limit {limit}")
    if exp['aqm'] == 'RED':
        router_node.execute(f"sudo tc qdisc replace dev {router_egress_name} parent 1:3 handle 3: red limit {limit} avpkt 1000 bandwidth {exp['rate']}Mbit")

    # Remove existing result files from hosts
        
    # Set up delay on receiver interface
    for n in receiver_nodes:
        receiver_inf_name = receiver_ingress_names[0]
        n.execute(f"sudo tc qdisc del dev {receiver_inf_name} root netem")
        n.execute(f"sudo tc qdisc add dev {receiver_inf_name} root netem delay {exp['btl_delay']}ms limit 1000000")
        
        
    
        
def add_sender_link_delays(sender_nodes, trial):
    for i, n in enumerate(sender_nodes):
        sender_inf_name = sender_egress_names[i]

        sender_delay = random.uniform(20, 30) # random_delays[f'Trial {trial}'][i]


        #print(sender_delay)

        n.execute(f"sudo tc qdisc del dev {sender_inf_name} root")
        n.execute(f"sudo tc qdisc add dev {sender_inf_name} root netem delay {sender_delay}ms limit 1000000")
        # n.execute(f"sudo tc qdisc add dev {sender_inf_name} parent 1:1 handle 10: fq pacing")

        
def start_servers(sender_nodes, base_port):  
    for i,s in enumerate(sender_nodes):
        s.execute("sudo killall iperf3")
        
        server_port = base_port + i
        s.execute(f"iperf3 -s -D -p {server_port}")

def start_clients(receiver_node, sender_nodes, exp, base_port):
    print("Start parallel clients on the senders")
    
    r = receiver_node
    r.execute("rm *.json")
    r.execute("sudo killall /home/ubuntu/iperf/src/iperf3")
    
    #print(router_script.format(iface=router_egress_name, duration=exp["test_duration"]))
    
    
    for i, n in enumerate(sender_nodes):
        
        server_port = base_port + i
        server_id = 10 + i
        
        cca = exp['cca'].split("-")[0] if i < 5 else exp['cca'].split("-")[1]
        
        #print(f'sleep 1; iperf3 -c 10.10.2.10 -t {exp["test_duration"]} -P {exp["flows"]} -C {cca} -p {server_port} -O {exp["omit"]} -i {exp["interval"]} -J > flow-{i}-result.json')

        r.execute_thread(f'sleep 1; /home/ubuntu/iperf/src/iperf3 -c 10.10.1.{server_id} -t {exp["test_duration"]} -P {exp["flows"]} -C {cca} -p {server_port} -O {exp["omit"]} -i {exp["interval"]} -R -J > flow-{i}-result.json')
        
    router_node.execute_thread(router_script.format(iface=router_egress_name, duration=exp["test_duration"]))
    
def copy_files_to_dir(exp_dir):   
    
    receiver_nodes[0].execute(f'mkdir -p {exp_dir} && mv *.json {exp_dir}/')
    
    router_node.execute(f'mkdir -p {exp_dir} && mv *.txt {exp_dir}/')
    
    router_node.execute(f'mkdir -p {exp_dir} && mv *.csv {exp_dir}/')
        
def compute_jfi(throughput):

    #mean_throughput = sum(throughput) / len(throughput)

    denominator = sum(t ** 2 for t in throughput)
    
    numerator = (sum(throughput)) ** 2
    
    jfi = (numerator / (len(throughput) * denominator))
    
    return jfi
        
def process_data(receiver_node, sender_nodes, exp):
    
    r = receiver_node
    
    sum_tput = 0
    
    pkt_loss = 0
    
    throughput = []
    for i,s in enumerate(sender_nodes):
        
        sum_received, stderr = r.execute(f'echo $(jq \'.end.sum_received.bits_per_second\' \"flow-{i}-result.json\")');
        
        throughput.append(float(sum_received))
        
        retrans, stderr = r.execute(f'echo $(jq \'.end.streams[].sender.retransmits\' \"flow-{i}-result.json\")')
        
        retrans = float(retrans)
        pkt_loss = pkt_loss + retrans
        
    loss_percentage = ((pkt_loss * 1500 * 8) * 100) / (sum(throughput) * exp['test_duration'])
    
    utilization = (sum(throughput) * 100)/ (exp['rate'] * 1000000)
        
    jfi = compute_jfi(throughput)
    
    
    router_node.execute("cat queue.txt | awk '{{print $1\",\"$24$30\",\"$31}}' | tr -d 'b' | tr -d 'p' | sed 's/K/000/' | sed 's/M/000000/' > queue.csv")
    
    mean_backlog, stderr = router_node.execute("echo $(awk -F ',' '{ total += $3; count++ } END { print total/count }' \"queue.csv\")")
    
    limit = math.ceil(((exp['buffer_factor']+1)*exp['rate'] * exp['btl_delay'] * 10**3)/8)
    
    buffer_occupancy = (float(mean_backlog) * 100)/limit
    
    
#     print("Jain's fairness index = ", jfi)
    
#     print("Loss percentage = ", loss_percentage)
    
#     print("Utilization = ", utilization)
    
#     print("Buffer occupancy percentage = ", buffer_occupancy)
    
    row = {**exp, 'jfi': jfi, 'loss_percentage': loss_percentage, 'utilization': utilization, 'buffer_occupancy': buffer_occupancy}
    with open('experiment_results.csv', mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fields)
        writer.writerow(row)

        
    
for exp in exp_lists:
    
    setup_experiment(exp)
    
    add_sender_link_delays(sender_nodes, exp['trial'])
    
    base_port = 5201

    # Start servers
    start_servers(sender_nodes, base_port)
    
    # Start clients
    start_clients(receiver_nodes[0], sender_nodes, exp, base_port)
    
    time.sleep(exp['test_duration'] + exp['omit'] + 10)
    
    
    process_data(receiver_nodes[0], sender_nodes, exp)
    
    
    # Copy files to directory
    exp_dir = f"scherrer-exp-aggregate/{'_'.join(str(v) for v in exp.values())}"
    copy_files_to_dir(exp_dir)



In [ ]:
f'sleep 1; /home/ubuntu/iperf/src/iperf3 -c 10.10.1.10 -t {exp["test_duration"]} -P {exp["flows"]} -C bbr2 -p 5201 -O {exp["omit"]} -i {exp["interval"]} -R -J > flow-0-result.json'

In [ ]:
home/ubuntu/iperf/src/iperf3 -c 10.10.1.10 -t 300 -P 1 -C bbr2 -p 5201 -O 180 -i 0.1 -R -J > flow-{i}-result.json

In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('experiment_results.csv')

# Perform the operation: Multiply buffer occupancy value by (buffer_factor + 1) / buffer_factor
df['buffer_occupancy'] = df['buffer_occupancy'] * (df['buffer_factor'] + 1) / df['buffer_factor']

# Save the modified DataFrame back to a CSV file
df.to_csv('modified_experiment_results.csv', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a DataFrame
data = pd.read_csv('experiment_results_final.csv')

# Group data by 'aqm' column
grouped_by_aqm = data.groupby('aqm')

# Define the metrics you want to plot
metrics = ['jfi', 'loss_percentage', 'utilization', 'buffer_occupancy']

# Define markers for each CCA combination
markers = ['o', 's', '^', 'D', 'x', 'P', '*', 'v', '<', '>']

# Create subplots with 2 columns
num_metrics = len(metrics)
num_rows = num_metrics // 2 + num_metrics % 2  # Adjust for odd number of metrics
fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(12, 4 * num_rows))

# Flatten axes for easy iteration
axes = axes.flatten()

# Iterate through each AQM type
for aqm, aqm_group in grouped_by_aqm:
    # Iterate through each metric
    for idx, metric in enumerate(metrics):
        ax = axes[idx]  # Select the current subplot
        
        ax.set_title(f"AQM Type: {aqm} - {metric}")
        ax.set_xlabel('Buffer Size (in BDP)')
        ax.set_ylabel(metric)
        
        # Group data by 'cca' column
        grouped_data = aqm_group.groupby('cca')
        
        # Iterate through each CCA combination
        for i, (cca, group) in enumerate(grouped_data):
            # Group data by 'buffer_factor' and calculate the mean for each group
            mean_data = group.groupby('buffer_factor')[metric].mean()
            
            # Plot the data for this CCA combination with marker
            ax.plot(mean_data.index, mean_data.values, marker=markers[i % len(markers)], label=f'CCA {cca}')
        
        # Set Y limits for each metric
        if metric == 'jfi':
            ax.set_ylim(0, 1)
        elif metric == 'loss_percentage':
            ax.set_ylim(0, 25)
        elif metric == 'utilization':
            ax.set_ylim(90, 102)
        elif metric == 'buffer_occupancy':
            ax.set_ylim(0, 100)
    
        ax.grid(True)

# Create a single legend for all subplots on the right side
axes[0].legend(bbox_to_anchor=(1.1, 1), loc='upper left')

# Adjust layout
plt.tight_layout()

# Save plots to PDF
plt.savefig('experiment_results.pdf', format='pdf')

plt.show()


In [ ]:


# Read the two CSV files
df1 = pd.read_csv('modified_experiment_results.csv')
df2 = pd.read_csv('modified_experiment_results_2.csv')

# Concatenate the DataFrames vertically
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('experiment_results_final.csv', index=False)

In [ ]:

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('experiment_results_final.csv')

# Apply the condition: If utilization exceeds 100, cut it to 100
df['utilization'] = df['utilization'].clip(upper=100)

# Save the modified DataFrame back to a CSV file
df.to_csv('experiment_results_final.csv', index=False)


In [ ]:
import random

# Create an empty dictionary to store the sets of random numbers
random_delays = {}

# Generate and store random numbers for three trials
for trial_number in range(1, 4):
    random_set = [random.uniform(20, 30) for _ in range(10)]  # Generating 5 random numbers between 20 and 30
    random_delays[f"Trial {trial_number}"] = random_set

# Print the dictionary containing the random numbers
print(random_delays)